In [ ]:
import os
import re
import cv2
from matplotlib import pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from concurrent.futures import ProcessPoolExecutor, as_completed

# Step 0: Sort & Rename images

In [4]:

def natural_sort_key(s):
    # Extract numbers and return as a sorting key
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def list_image_files(directory):
    # List to store image file names
    image_files = []

    # Walk through directory and subdirectories
    for root, _, files in os.walk(directory):
        for file in files:
            image_files.append(os.path.join(file))
            
    image_files.sort(key=natural_sort_key)
    return image_files

def rename_files(directory):
    day = input("day: ")
    temp = input("temp: ")
    rep = 1
    pic = 1
    
    # Get the sorted list of image files
    image_files = list_image_files(directory)
    
    for file in image_files:
        print(file)
        if pic == 3:
            pic = 1
            rep += 1
        os.rename(directory + "/" + file, directory+ "/" + f"a_{day}_{temp}_{str(rep)}_{str(pic)}" + '.jpg')
        pic += 1
    return None


In [ ]:
directory_path = "../raw_images/Day 8_24 Jan 2023/15C"
rename_files(directory_path)
# image_files = list_image_files(directory_path)


# Step 1: Image Normalization

## Traditional Technique

### Plot Original Image

In [ ]:
# Load the image
image = cv2.imread('../raw_images/0_5_1_1.JPG')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for visualization
plt.imshow(image_rgb)
plt.title("Original Image")
plt.show()

### Convert to Gray

In [ ]:
# Convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Normalize the pixel values to range [0, 1]
normalized_image = gray_image / 255.0

plt.imshow(normalized_image, cmap='gray')
plt.title("Preprocessed Image")
plt.show()


### Create Mask Area

In [ ]:
# Thresholding to create a binary mask
_, binary_mask = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY_INV)

# Morphological operations to clean up the mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
cleaned_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)

plt.imshow(cleaned_mask, cmap='gray')
plt.title("Binary Mask")
plt.show()

In [ ]:
# Extract the leaf area using the mask
leaf_area = cv2.bitwise_and(image, image, mask=cleaned_mask)

plt.imshow(leaf_area)
plt.title("Leaf Area")
plt.show()


In [ ]:
# Normalize the target area for better focus
leaf_area_normalized = leaf_area / 255.0

plt.imshow(leaf_area_normalized)
plt.title("Normalized Leaf Area")
plt.show()

In [ ]:



image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert to HSV color space
image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define green color range (adjust based on your image)
lower_green = np.array([30, 40, 40])  # Hue, Saturation, Value
upper_green = np.array([90, 255, 255])

# Create a mask for green regions
mask_green = cv2.inRange(image_hsv, lower_green, upper_green)

# Apply the mask to the original image
green_area = cv2.bitwise_and(image_rgb, image_rgb, mask=mask_green)

plt.imshow(green_area)
plt.title("Green Area (HSV Segmentation)")
plt.show()
cv2.imwrite("hsv_green_area.jpg", cv2.cvtColor(green_area, cv2.COLOR_BGR2RGB))


In [ ]:
# Convert to grayscale for edge detection
gray_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)

# Apply GaussianBlur to reduce noise
blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# Apply Canny edge detection
edges = cv2.Canny(blurred_image, threshold1=50, threshold2=150)

plt.imshow(edges, cmap='gray')
plt.title("Edge Detection (Canny)")
plt.show()

## R-CNN

In [ ]:
import torch
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
import numpy as np
import os

# Function to segment and extract leaves using Mask R-CNN
def extract_leaves_with_maskrcnn(image_path, output_path, threshold=0.5):
    # Load pretrained Mask R-CNN model
    model = maskrcnn_resnet50_fpn(pretrained=True)
    model.eval()

    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    image_tensor = F.to_tensor(image).unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        predictions = model(image_tensor)

    # Extract masks, scores, and labels
    masks = predictions[0]['masks'].squeeze().cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()

    # Combine masks for objects with high confidence
    final_mask = np.zeros_like(masks[0], dtype=np.uint8)
    for i in range(len(scores)):
        if scores[i] >= threshold and labels[i] == 15:  # Class '15' = person; replace for custom class
            final_mask = np.maximum(final_mask, (masks[i] > 0.5).astype(np.uint8))

    # Apply the mask to the original image
    image_np = np.array(image)
    result = image_np * np.expand_dims(final_mask, axis=2)

    # Save the result
    result_image = Image.fromarray(result.astype('uint8'))
    result_image.save(output_path)
    print(f"Extracted leaf saved to {output_path}")

# Process a folder of images
def process_folder_with_maskrcnn(input_folder, output_folder, threshold=0.5):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"extracted_{filename}")
            extract_leaves_with_maskrcnn(input_path, output_path, threshold)
# Example usage
input_folder = "../raw_images"  # Replace with your folder path
output_folder = "../output/r_cnn"  # Replace with your desired output folder path

process_folder_with_maskrcnn(input_folder, output_folder, threshold=0.5)

In [ ]:
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from torchvision.transforms import functional as F
import torch

# Load your dataset
class CustomDataset(CocoDetection):
    def __init__(self, img_folder, ann_file, transforms=None):
        super(CustomDataset, self).__init__(img_folder, ann_file)
        self.transforms = transforms

    def __getitem__(self, idx):
        img, target = super(CustomDataset, self).__getitem__(idx)
        if self.transforms:
            img = self.transforms(img)
        return img, target

# Define the training function
def train_model(dataset_path, annotation_path, model_save_path):
    # Load dataset
    dataset = CustomDataset(
        img_folder=dataset_path,
        ann_file=annotation_path,
        transforms=F.to_tensor
    )
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

    # Load Mask R-CNN model
    model = maskrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2  # Background and leaf
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

    # Define optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    # Train the model
    model.train()
    for epoch in range(10):  # Number of epochs
        for imgs, targets in dataloader:
            imgs = [img for img in imgs]
            loss_dict = model(imgs, targets)
            losses = sum(loss for loss in loss_dict.values())
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            print(f"Epoch {epoch}, Loss: {losses.item()}")

    # Save the model
    torch.save(model, model_save_path)
    print(f"Model saved to {model_save_path}")

# Train the model
train_model(
    dataset_path="/path/to/your/images",
    annotation_path="/path/to/your/annotations.json",
    model_save_path="/path/to/save/model.pth"
)


## Gray Extract

In [ ]:
import cv2
import numpy as np
import os

def extract_leaves_white_bg(image_path, output_path):
      # Load the image
    image = cv2.imread(image_path)
    original = image.copy()

    # Convert the image to grayscale and HSV
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply adaptive thresholding for general segmentation
    adaptive_mask = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Define color ranges for green and yellow leaves
    lower_green = np.array([25, 40, 40])  # Green color range
    upper_green = np.array([90, 255, 255])

    lower_yellow = np.array([20, 100, 100])  # Yellow color range
    upper_yellow = np.array([40, 255, 255])

    # Create color masks
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Combine color masks
    color_mask = cv2.bitwise_or(green_mask, yellow_mask)

    # Combine with adaptive thresholding mask
    combined_mask = cv2.bitwise_and(color_mask, adaptive_mask)

    # Refine mask with morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    refined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_OPEN, kernel)

    # Extract the leaves by applying the refined mask
    result = cv2.bitwise_and(original, original, mask=refined_mask)

    # Save the result
    cv2.imwrite(output_path, result)
    print(f"Processed and saved: {output_path}")

# Process a folder of images
def process_folder_white_bg(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{filename}")
            extract_leaves_white_bg(input_path, output_path)

# Example usage
input_folder = "../raw_images"  # Replace with your folder containing input images
output_folder = "../output/gray_extract"  # Replace with the desired output folder

process_folder_white_bg(input_folder, output_folder)



In [ ]:
import cv2
import numpy as np
import os

def extract_leaves_with_gaussian(image_path, output_path):
    # Load the image
    image = cv2.imread(image_path)
    original = image.copy()

    # Convert the image to grayscale and HSV
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply Gaussian blur to smooth the image
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)

    # Apply adaptive thresholding for general segmentation
    adaptive_mask = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Define color ranges for green and yellow leaves
    lower_green = np.array([25, 40, 40])  # Green color range
    upper_green = np.array([90, 255, 255])

    lower_yellow = np.array([20, 100, 100])  # Yellow color range
    upper_yellow = np.array([40, 255, 255])

    # Create color masks
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Combine color masks
    color_mask = cv2.bitwise_or(green_mask, yellow_mask)

    # Combine with adaptive thresholding mask
    combined_mask = cv2.bitwise_and(color_mask, adaptive_mask)

    # Apply Gaussian blur to refine the combined mask
    refined_mask = cv2.GaussianBlur(combined_mask, (5, 5), 0)

    # Further refine with morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_CLOSE, kernel)
    refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_OPEN, kernel)

    # Extract the leaves by applying the refined mask
    result = cv2.bitwise_and(original, original, mask=refined_mask)

    # Save the result
    cv2.imwrite(output_path, result)
    print(f"Processed and saved: {output_path}")

# Process a folder of images
def process_folder_with_gaussian(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"extracted_{filename}")
            extract_leaves_with_gaussian(input_path, output_path)

# Example usage
input_folder = "../raw_images"  # Replace with your folder containing input images
output_folder = "../output/gassian"  # Replace with the desired output folder

process_folder_with_gaussian(input_folder, output_folder)


In [1]:
import cv2
import numpy as np
import os
import logging
from multiprocessing import Pool

In [ ]:

# Configure logging for the main process
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()],
    force=True
)

# Function to configure logging in each process
def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[logging.StreamHandler()]
    )

# Function to process a single image
def process_image(args):
    input_path, output_path = args
    setup_logging()  # Ensure logging is set up for this child process
    try:
        logging.info(f"Processing image: {input_path}")

        # Load the image
        image = cv2.imread(input_path)
        if image is None:
            logging.error(f"Failed to load image: {input_path}")
            return

        logging.info("Image loaded successfully.")

        # Step 1: Resize the image to speed up processing (scale to 50%)
        height, width = image.shape[:2]
        scale_factor = 0.5
        image_resized = cv2.resize(image, (int(width * scale_factor), int(height * scale_factor)))
        logging.info(f"Image resized to {image_resized.shape[:2]} for faster processing.")

        # Step 2: Define the rectangle for GrabCut (tight bounding box)
        resized_height, resized_width = image_resized.shape[:2]
        rect = (10, 10, resized_width - 20, resized_height - 20)
        logging.info(f"Defined rectangle for GrabCut: {rect}")

        # Step 3: Initialize GrabCut mask and models
        mask = np.zeros(image_resized.shape[:2], np.uint8)
        bgd_model = np.zeros((1, 65), np.float64)
        fgd_model = np.zeros((1, 65), np.float64)
        logging.info("Initialized GrabCut models.")

        # Step 4: Run GrabCut with fewer iterations
        logging.info("Starting GrabCut process...")
        cv2.grabCut(image_resized, mask, rect, bgd_model, fgd_model, 2, cv2.GC_INIT_WITH_RECT)
        logging.info("GrabCut process completed.")

        # Step 5: Convert GrabCut mask to binary format
        mask_binary = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        logging.info("Converted mask to binary format.")

        # Step 6: Extract the foreground (leaves) from the resized image
        foreground = image_resized * mask_binary[:, :, np.newaxis]

        # Step 7: Resize back to the original resolution
        result = cv2.resize(foreground, (width, height))
        logging.info("Resized result back to original dimensions.")

        # Save the result
        cv2.imwrite(output_path, result)
        logging.info(f"Processed image saved to: {output_path}")

    except Exception as e:
        logging.error(f"Error processing image {input_path}: {str(e)}")

# Function to process a folder of images
def process_folder_grabcut(input_folder, output_folder):
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            logging.info(f"Created output folder: {output_folder}")

        # Get all image files
        image_files = [
            (os.path.join(input_folder, f), os.path.join(output_folder, f"grabcut_{f}"))
            for f in os.listdir(input_folder)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]

        # Use multiprocessing for parallel processing
        with Pool() as pool:  # Adjust the number of processes based on your CPU cores
            pool.map(process_image, image_files)

        logging.info("All images processed successfully.")

    except Exception as e:
        logging.error(f"Error processing folder: {str(e)}")

# Example usage
input_folder = "../raw_images" 
output_folder = "../output/grapcut"

process_folder_grabcut(input_folder, output_folder)
